In [1]:
from pandas import Series, DataFrame
import pickle
import numpy as np
from collections import defaultdict
import pandas as pd
df_s = pd.read_csv('E:/Charlotte/CS/MACHINE LEARNING/RECOMMENDATION/Amazon_Instant_Video_noreview/Amazon_Instant_Video_noreview.csv', sep=',')
file_path_s = 'E:/Charlotte/CS/MACHINE LEARNING/RECOMMENDATION/Amazon_Instant_Video_noreview/Amazon_Instant_Video_noreview.csv'
path_s = '/'.join(file_path_s.split('/')[:-1])
name_s = file_path_s.split('/')[-1].split('_')[0]
train_npy_path_s = file_path_s.replace('.csv', '_train.npy')
test_npy_path_s = file_path_s.replace('.csv', '_test.npy')
def _load_data(df):
    uuu=0
    iii=0
    for user, item in zip(df['uid'], df['iid']):
        if uuu < user:
            uuu = user
        if iii < item:
            iii = item        
    return uuu+1,iii+1
num_users_s, num_items_s = _load_data(df_s)
print("num_users_s",num_users_s)
print("num_items_s",num_items_s)
def _construct_pos_dict(df):
    pos_dict = defaultdict(set)
    for user, item in zip(df['uid'], df['iid']):
        pos_dict[user].add(item)
    return pos_dict
pos_dict_s=_construct_pos_dict(df_s)
def read_data(dataPath):
    inFile = open(dataPath + 'data_HGNR.pkl','rb')
    data = pickle.load(inFile)
    userNum, itemNum = data['userNum'], data['itemNum']
    trainData, testData, testNegData  = data['trainData'], data['testData'], data['testNegData']
    userUserSeq1Matrix,itemItemSeq1Matrix = data['userUserSeq1Matrix'], data['itemItemSeq1Matrix']
    inFile.close()
    return trainData, testData, testNegData,userUserSeq1Matrix,itemItemSeq1Matrix,userNum, itemNum
dataPath1 = 'E:/Charlotte/CS/MACHINE LEARNING/RECOMMENDATION/Amazon_Instant_Video_noreview/'
trainData_s, testData_s,testNegData_s,userUserSeq1Matrix,itemItemSeq1Matrix, userNum_s, itemNum_s = read_data(dataPath1)
print('userNum_s',userNum_s)
print('itemNum_s',itemNum_s)
train_df_s1,train_df_s2 = [],[]
test_df_s1,test_df_s2 = [],[]
train_df_s,test_df_s = {'uid':[],'iid':[]},{'uid':[],'iid':[]}
train_df_s,test_df_s =  pd.DataFrame(train_df_s),pd.DataFrame(test_df_s)
for u,i in trainData_s:   
    train_df_s1.append(u)
    train_df_s2.append(i)
train_df_s['uid'] = train_df_s1
train_df_s['iid'] = train_df_s2 
train_df_s.to_csv(path_s+'/%s_train_df.csv'% name_s, index=False)
for u,i in testData_s:
    test_df_s1.append(u)
    test_df_s2.append(i)
test_df_s['uid'] = test_df_s1
test_df_s['iid'] = test_df_s2 
test_df_s.to_csv(path_s+'/%s_test_df.csv'% name_s, index=False)
def _add_negtive(user, item, num_items, pos_dict, neg_num, boolindex):
    user, item, num_items, pos_dict, neg_num, train = user, item, num_items, pos_dict,neg_num ,boolindex
    users, items, labels = [], [], []
    neg_set = set(range(num_items)).difference(pos_dict[user])  #difference用于返回集合的差集，包含在第一个集合中，但不包含在第二个集合中#
    neg_sample_list = np.random.choice(list(neg_set), neg_num, replace=False).tolist()
    for neg_sample in neg_sample_list:
        users.append(user)
        items.append(neg_sample)
        labels.append(0) if train == True else labels.append(neg_sample)
    users.append(user)
    items.append(item)
    if train == True:
        labels.append(1)
    else:
        labels.append(int(item))
    return (users, items, labels)

users = []
items = []
labels = []    
for user, item in zip(train_df_s['uid'], train_df_s['iid']):
    batch_users, batch_items, batch_labels = _add_negtive(user, item, num_items_s, pos_dict_s,4 ,True)
    users += batch_users
    items += batch_items
    labels += batch_labels
"""
str --> int
results = ['1','2','3']
results = list(map(int, results))    
"""
users = list(map(int, users))
items = list(map(int, items))
labels = list(map(int, labels))
data_dict_str = {'user': users, 'item': items, 'label': labels}
np.save(train_npy_path_s, data_dict_str)
users = []
items = []
labels = []    
for user, item in zip(test_df_s['uid'], test_df_s['iid']):
    batch_users, batch_items, batch_labels = _add_negtive(user, item, num_items_s, pos_dict_s,99 ,False)
    users += batch_users
    items += batch_items
    labels += batch_labels
users = list(map(int, users))
items = list(map(int, items))
labels = list(map(int, labels))
data_dict_ste = {'user': users, 'item': items, 'label': labels}
np.save(test_npy_path_s, data_dict_ste)
print("ok")





num_users_s 5130
num_items_s 1685
userNum_s 5130
itemNum_s 1685
ok


In [2]:
import numpy as np
import tensorflow as tf
import argparse
import sys, os
sys.path.append("..")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
parser = argparse.ArgumentParser()
parser.add_argument('--gpu_device', type=int, default=0,
                    help='choose which gpu to run')
parser.add_argument('--cross_data_rebuild', type=bool, default=False,
                    help='whether to rebuild cross data')
parser.add_argument('--data_rebuild', type=bool, default=False,
                    help='whether to rebuild train/test dataset')
parser.add_argument('--mat_rebuild', type=bool, default=False,
                    help='whether to rebuild` adjacent mat')
parser.add_argument('--processor_num', type=int, default=12,
                    help='number of processors when preprocessing data')
parser.add_argument('--batch_size', type=int, default=128,
                    help='size of mini-batch')
parser.add_argument('--train_neg_num', type=int, default=4,
                    help='number of negative samples per training positive sample')
parser.add_argument('--test_size', type=int, default=1,
                    help='size of sampled test data')
parser.add_argument('--test_neg_num', type=int, default=99,
                    help='number of negative samples for test')
parser.add_argument('--epochs', type=int, default=150,
                    help='the number of epochs')
parser.add_argument('--gnn_layers', nargs='?', default=[64,64,64],
                    help='the unit list of layers')
parser.add_argument('--mlp_layers', nargs='?', default=[32,16,8],
                    help='the unit list of layers')
parser.add_argument('--embedding_size', type=int, default=64,
                    help='the size for embedding user and item')
parser.add_argument('--topK', type=int, default=10,
                    help='topk for evaluation')
parser.add_argument('--regularizer_rate', type=float, default=8e-5,   ####default=0.01
                    help='the regularizer rate')
parser.add_argument('--lr', type=float, default=0.005,
                    help='learning rate')
parser.add_argument('--dropout_message', type=float, default=0,  ####default=0.1
                    help='dropout rate of message')
parser.add_argument('--NCForMF', type=str, default='MF',
                    help='method to propagate embeddings')
args = parser.parse_args(args=[])
print(args)

import tensorflow
tensorflow.__version__
import tensorflow as tf
import numpy as np
import math
import numpy as np
import scipy.sparse as sp
import os, sys, time
from tqdm import tqdm
sys.path.append("..")
from multiprocessing import Pool
import tensorflow as tf
# from utils import metrics
class ReviewGCN(object):
    def __init__(self, args, iterator, norm_adj_mat1, norm_adj_mat2, num_users, num_items_s,is_training):
        self.args = args
        self.iterator = iterator
        self.norm_adj_mat1 = norm_adj_mat1
        self.norm_adj_mat2 = norm_adj_mat2
        self.num_users = num_users
        self.num_items_s = num_items_s
        self.is_training = is_training
        self.n_fold = 50
        self.get_data()
        self.all_weights = self.init_weights()
        self.item_embeddings_s1, self.user_embeddings1 = self.create_lightgcn_embed1()
        self.item_embeddings_s2, self.user_embeddings2 = self.create_lightgcn_embed2()
        self.item_embeddings_s = self.item_embeddings_s1 + 0.1 * self.item_embeddings_s2
        self.user_embeddings = self.user_embeddings1 + 0.1 * self.user_embeddings2

        self.inference()
        self.saver = tf.train.Saver(tf.global_variables())
    def get_data(self):
        sample = self.iterator.get_next()
        self.user, self.item_s= sample['user'], sample['item']
        self.label_s = tf.cast(sample['label'], tf.float32)
    def init_weights(self):
        all_weights = dict()
        initializer = tf.truncated_normal_initializer(0.01)
        regularizer = tf.contrib.layers.l2_regularizer(self.args.regularizer_rate)
        all_weights['user_embeddings1'] = tf.get_variable(
            'user_embeddings1', (self.num_users, self.args.embedding_size), tf.float32, initializer, regularizer)
        all_weights['item_embeddings_s1'] = tf.get_variable(
            'item_embeddings_s1', (self.num_items_s, self.args.embedding_size), tf.float32, initializer, regularizer)
        all_weights['user_embeddings2'] = tf.get_variable(
            'user_embeddings21', (self.num_users, self.args.embedding_size), tf.float32, initializer, regularizer)
        all_weights['item_embeddings_s2'] = tf.get_variable(
            'item_embeddings_s21', (self.num_items_s, self.args.embedding_size), tf.float32, initializer, regularizer)
        self.layers_plus = [self.args.embedding_size] + self.args.gnn_layers
        print("self.layers_plus",self.layers_plus)
        
        for k in range(len(self.layers_plus)-1):
            all_weights['W_gc1_%d' % k] = tf.get_variable(
                'W_gc1_%d'% k, (self.layers_plus[k], self.layers_plus[k+ 1]), tf.float32, initializer, regularizer)
            all_weights['b_gc1_%d' % k] = tf.get_variable(
                'b_gc1_%d'% k, self.layers_plus[k+ 1], tf.float32, tf.zeros_initializer(), regularizer)
            all_weights['W_gc2_%d' % k] = tf.get_variable(
                'W_gc2_%d'% k, (self.layers_plus[k], self.layers_plus[k+ 1]), tf.float32, initializer, regularizer)
            all_weights['b_gc2_%d' % k] = tf.get_variable(
                'b_gc2_%d'% k, self.layers_plus[k+ 1], tf.float32, tf.zeros_initializer(), regularizer)            
            
            all_weights['W_mlp_%d' % k] = tf.get_variable(
                'W_mlp_%d'% k, (self.layers_plus[k + 1], self.layers_plus[k + 1]), tf.float32, initializer, regularizer)
            all_weights['b_mlp_%d' % k] = tf.get_variable(
                'b_mlp_%d'% k, self.layers_plus[k+ 1], tf.float32, tf.zeros_initializer(), regularizer)
        return all_weights
    #####################GCNGCNGCNGCNGCNGCNGCNGCNGCNGCN#############################################   
    def creat_gcn_embedd1(self):
        A_fold_hat = self._split_A_hat(self.norm_adj_mat1)
        embeddings = tf.concat([self.all_weights['item_embeddings_s1'], self.all_weights['user_embeddings1']], axis=0)
        all_embeddings = [embeddings]
        for k in range(len(self.layers_plus)-1):
            temp_embedd = [tf.sparse_tensor_dense_matmul(A_fold_hat[f], embeddings) for f in range(self.n_fold)]
            embeddings = tf.concat(temp_embedd, axis=0)
            embeddings = tf.nn.leaky_relu(tf.matmul(embeddings, self.all_weights['W_gc1_%d'%k])
                                          + self.all_weights['b_gc1_%d'%k])
            embeddings = tf.nn.dropout(embeddings, 1 - self.args.dropout_message)
            all_embeddings += [embeddings]
        all_embeddings = tf.concat(all_embeddings, axis=1)
        item_embeddings_s, user_embeddings = tf.split(all_embeddings, [self.num_items_s, self.num_users], axis=0)
        return item_embeddings_s, user_embeddings
    def creat_gcn_embedd2(self):
        A_fold_hat = self._split_A_hat(self.norm_adj_mat2)
        embeddings = tf.concat([self.all_weights['item_embeddings_s2'], self.all_weights['user_embeddings2']], axis=0)
        all_embeddings = [embeddings]
        for k in range(len(self.layers_plus)-1):
            temp_embedd = [tf.sparse_tensor_dense_matmul(A_fold_hat[f], embeddings) for f in range(self.n_fold)]
            embeddings = tf.concat(temp_embedd, axis=0)
            embeddings = tf.nn.leaky_relu(tf.matmul(embeddings, self.all_weights['W_gc2_%d'%k])
                                          + self.all_weights['b_gc2_%d'%k])
            embeddings = tf.nn.dropout(embeddings, 1 - self.args.dropout_message)
            all_embeddings += [embeddings]
        all_embeddings = tf.concat(all_embeddings, axis=1)
        item_embeddings_s, user_embeddings = tf.split(all_embeddings, [self.num_items_s, self.num_users], axis=0)
        return item_embeddings_s, user_embeddings  
    
    ########################lightGCNightGCNightGCNightGCN###############################################
    def create_lightgcn_embed1(self):
        A_fold_hat = self._split_A_hat(self.norm_adj_mat1)  #Arating
        ego_embeddings = tf.concat([self.all_weights['item_embeddings_s1'], self.all_weights['user_embeddings1']], axis=0)
        all_embeddings = [ego_embeddings]
        for k in range(len(self.layers_plus)-1):
            temp_embed = []
            for f in range(self.n_fold):
                temp_embed.append(tf.sparse_tensor_dense_matmul(A_fold_hat[f], ego_embeddings))
            side_embeddings = tf.concat(temp_embed, 0)
            ego_embeddings = side_embeddings
            all_embeddings += [ego_embeddings]
        all_embeddings=tf.stack(all_embeddings,1)
        all_embeddings=tf.reduce_mean(all_embeddings,axis=1,keepdims=False)
        item_embeddings_s, user_embeddings = tf.split(all_embeddings, [self.num_items_s, self.num_users], axis=0)
        return item_embeddings_s, user_embeddings
    def create_lightgcn_embed2(self):
        A_fold_hat = self._split_A_hat(self.norm_adj_mat2) #Areview 
        ego_embeddings = tf.concat([self.all_weights['item_embeddings_s2'], self.all_weights['user_embeddings2']], axis=0)
        all_embeddings = [ego_embeddings]
        for k in range(len(self.layers_plus)-1):
            temp_embed = []
            for f in range(self.n_fold):
                temp_embed.append(tf.sparse_tensor_dense_matmul(A_fold_hat[f], ego_embeddings))
            side_embeddings = tf.concat(temp_embed, 0)
            ego_embeddings = side_embeddings
            all_embeddings += [ego_embeddings]
        all_embeddings=tf.stack(all_embeddings,1)
        all_embeddings=tf.reduce_mean(all_embeddings,axis=1,keepdims=False)
        item_embeddings_s, user_embeddings = tf.split(all_embeddings, [self.num_items_s, self.num_users], axis=0)
        return item_embeddings_s, user_embeddings
    ################################################################################################# 
    def _split_A_hat(self, X):
        fold_len = math.ceil((X.shape[0]) / self.n_fold)
        A_fold_hat = [self._convert_sp_mat_to_sp_tensor( X[i_fold*fold_len :(i_fold+1)*fold_len])
                      for i_fold in range(self.n_fold)]

        return A_fold_hat
    def _convert_sp_mat_to_sp_tensor(self, X):
        coo = X.tocoo().astype(np.float32)
        indices = np.mat([coo.row, coo.col]).transpose()
        return tf.SparseTensor(indices, coo.data, coo.shape)
    def inference(self):      
        initializer = tf.truncated_normal_initializer(0.01)
        regularizer = tf.contrib.layers.l2_regularizer(self.args.regularizer_rate)
        with tf.name_scope('embedding'):
            user_embedding = tf.nn.embedding_lookup(self.user_embeddings, self.user)
            item_embedding_s = tf.nn.embedding_lookup(self.item_embeddings_s, self.item_s)
        with tf.name_scope('propagation'):
            if self.args.NCForMF == 'MF':
                self.logits_dense_s = tf.reduce_sum(tf.multiply(user_embedding, item_embedding_s), 1)  #logits_dense_s =256*1#
            elif self.args.NCForMF == 'NCF':
                a_s = tf.concat([user_embedding, item_embedding_s], axis=-1, name='inputs_s')
                for i, units in enumerate(self.args.mlp_layers):
                    dense_s = tf.layers.dense(a_s, units, tf.nn.relu, kernel_initializer=initializer,
                                          kernel_regularizer = regularizer, name='dense_s_%d' % i)
                    a_s = tf.layers.dropout(dense_s, self.args.dropout_message)
                self.logits_dense_s = tf.layers.dense(inputs=a_s,
                                                      units=1,
                                                      kernel_initializer=initializer,
                                                      kernel_regularizer=regularizer,
                                                      name='logits_dense_s')

            else:
                raise ValueError
            self.logits_s = tf.squeeze(self.logits_dense_s)

            loss_list_s = tf.nn.sigmoid_cross_entropy_with_logits(labels=self.label_s, logits=self.logits_s,     #交叉熵#
                                                                  name='loss_s')
            loss_w_s = tf.map_fn(lambda x: tf.cond(tf.equal(x, 1.0), lambda: 5.0, lambda: 1.0), self.label_s)

            self.loss_s = tf.reduce_mean(tf.multiply(loss_list_s, loss_w_s))
            self.loss = self.loss_s 
            self.optimizer = tf.train.AdamOptimizer(self.args.lr).minimize(self.loss)
            self.label_replica_s = self.label_s
            _, self.indice_s = tf.nn.top_k(tf.sigmoid(self.logits_s), self.args.topK)
    def step(self, sess):
        if self.is_training:
            label_s, indice_s, loss, optim = sess.run(
                [self.label_replica_s, self.indice_s, self.loss, self.optimizer])
            return loss
        else:
            label_s, indice_s = sess.run([self.label_replica_s, self.indice_s])
            prediction_s = np.take(label_s, indice_s)
            return prediction_s, label_s


D:\Anaconda\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:520: FutureWarning: Passing (type, 1)

Namespace(NCForMF='MF', batch_size=128, cross_data_rebuild=False, data_rebuild=False, dropout_message=0, embedding_size=64, epochs=150, gnn_layers=[64, 64, 64], gpu_device=0, lr=0.005, mat_rebuild=False, mlp_layers=[32, 16, 8], processor_num=12, regularizer_rate=8e-05, test_neg_num=99, test_size=1, topK=10, train_neg_num=4)


In [3]:
def metrics_mrr(gt_item, pred_items):
    if gt_item in pred_items:
        index = np.where(pred_items == gt_item)[0][0]
        return np.reciprocal(float(index + 1))
    else:
        return 0
def metrics_hit(gt_item, pred_items):
    if gt_item in pred_items:
        return 1
    return 0
def metrics_ndcg(gt_item, pred_items):
    if gt_item in pred_items:
        index = np.where(pred_items == gt_item)[0][0]
        return np.reciprocal(np.log2(index + 2))
    return 0
def evaluate(predictions, labels):
    label = int(labels[-1])
    hr = metrics_hit(label, predictions)
    mrr = metrics_mrr(label, predictions)
    ndcg = metrics_ndcg(label, predictions)
    return hr, mrr, ndcg

def normalized_adj_single(adj):
    rowsum = np.array(adj.sum(1))
    d_inv = np.power(rowsum, -1).flatten()
    d_inv[np.isinf(d_inv)] = 0.
    d_mat_inv = sp.diags(d_inv)
    norm_adj = d_mat_inv.dot(adj)
    return norm_adj

def load_mat_s1(num_users_s, num_items_s, data_dict_str,userUserSeq1Matrix,itemItemSeq1Matrix, args):
    print('Building adjacent mats_s..')  
    num_users = num_users_s
    num_items_s = num_items_s
    train_df_s = {'user':data_dict_str['user'][args.train_neg_num::args.train_neg_num+1],
                  'item':data_dict_str['item'][args.train_neg_num::args.train_neg_num+1]} 

    R_s = sp.dok_matrix((num_users, num_items_s), dtype=np.float32)
    U_s = sp.dok_matrix(userUserSeq1Matrix, dtype=np.float32)
    I_s = sp.dok_matrix(itemItemSeq1Matrix, dtype=np.float32)
    for user, item in zip(train_df_s['user'], train_df_s['item']):
        R_s[int(user), int(item)] = 1.0
    plain_adj_mat = sp.dok_matrix((num_items_s+ num_users, num_items_s+ num_users),
                                  dtype=np.float32).tolil()
    plain_adj_mat[num_items_s: num_items_s+ num_users, :num_items_s] = R_s
    plain_adj_mat[:num_items_s, num_items_s: num_items_s+ num_users] = R_s.T
    plain_adj_mat[:num_items_s, :num_items_s] =  0.0*I_s
    plain_adj_mat[num_items_s: num_items_s+ num_users, num_items_s: num_items_s+ num_users] =  0.0*U_s
    plain_adj_mat = plain_adj_mat.todok()
    norm_adj_mat_s1 = normalized_adj_single(plain_adj_mat+ sp.eye(plain_adj_mat.shape[0]))
    print('Get adjacent mats_s successfully.')
    return norm_adj_mat_s1
def load_mat_s2(num_users_s, num_items_s, data_dict_str,userUserSeq1Matrix,itemItemSeq1Matrix, args):
    print('Building adjacent mats_s..')  
    num_users = num_users_s
    num_items_s = num_items_s
    train_df_s = {'user':data_dict_str['user'][args.train_neg_num::args.train_neg_num+1],
                  'item':data_dict_str['item'][args.train_neg_num::args.train_neg_num+1]} 

    R_s = sp.dok_matrix((num_users, num_items_s), dtype=np.float32)
    U_s = sp.dok_matrix(userUserSeq1Matrix, dtype=np.float32)
    I_s = sp.dok_matrix(itemItemSeq1Matrix, dtype=np.float32)
    for user, item in zip(train_df_s['user'], train_df_s['item']):
        R_s[int(user), int(item)] = 1.0
    plain_adj_mat = sp.dok_matrix((num_items_s+ num_users, num_items_s+ num_users),
                                  dtype=np.float32).tolil()
    plain_adj_mat[num_items_s: num_items_s+ num_users, :num_items_s] = 0.0*R_s
    plain_adj_mat[:num_items_s, num_items_s: num_items_s+ num_users] = 0.0*R_s.T
    plain_adj_mat[:num_items_s, :num_items_s] =  0.1*I_s
    plain_adj_mat[num_items_s: num_items_s+ num_users, num_items_s: num_items_s+ num_users] =  0.1*U_s
    plain_adj_mat = plain_adj_mat.todok()
    norm_adj_mat_s2 = normalized_adj_single(plain_adj_mat+ sp.eye(plain_adj_mat.shape[0]))
    print('Get adjacent mats_s successfully.')
    return norm_adj_mat_s2
##################################################################################################
if __name__ == '__main__':
    with tf.Session() as sess:
        norm_adj_mat1 = load_mat_s1(num_users_s, num_items_s, data_dict_str,userUserSeq1Matrix,itemItemSeq1Matrix, args)
        norm_adj_mat2 = load_mat_s2(num_users_s, num_items_s, data_dict_str,userUserSeq1Matrix,itemItemSeq1Matrix, args)
        train_data = tf.data.Dataset.from_tensor_slices(data_dict_str)
        train_data = train_data.shuffle(buffer_size=len(data_dict_str['user'])).batch(args.batch_size)
        print('train_data',train_data)
        print('train_data除以batch_size',len(data_dict_str['user'])/args.batch_size)
        test_data = tf.data.Dataset.from_tensor_slices(data_dict_ste)
        print('test_data',test_data) 
        test_data = test_data.batch(args.test_size + args.test_neg_num)

        iterator = tf.data.Iterator.from_structure(train_data.output_types, train_data.output_shapes)
        model = ReviewGCN(args, iterator, norm_adj_mat1, norm_adj_mat2, num_users_s, num_items_s, True)
        print("Creating model with fresh parameters...")
        sess.run(tf.global_variables_initializer())

        count = 0
        loss = 0
        last_count = 0
        hr_s_list, mrr_s_list, ndcg_s_list = [], [], []
        for epoch in range(1, args.epochs + 1):
            print('=' * 30 + ' EPOCH %d ' % epoch + '=' * 30)
            ################################## TRAINING ################################
            if 6 > epoch > 3:
                model.args.lr = 1e-3
            if epoch >= 6:
                model.args.lr = 1e-4
            sess.run(model.iterator.make_initializer(train_data))
            model.is_training = True
            start_time = time.time()
            try:
                while True:
                    count += 1
                    loss += model.step(sess)
                    if count % 300 == 0:
                        print('Epoch %d, step %d, with average loss of %.4f in last %d steps;'
                              % (epoch, count, loss / (count - last_count), count - last_count))
                        loss = 0
                        last_count = count
            except tf.errors.OutOfRangeError:
                print("Epoch %d, finish training " % epoch + "took " +
                      time.strftime("%H: %M: %S", time.gmtime(time.time() - start_time)) + ';')

            ################################## TESTING ################################
            sess.run(model.iterator.make_initializer(test_data))
            model.is_training = False
            start_time = time.time()
            HR_s, MRR_s, NDCG_s = [], [], []
            predictions_s, labels_s = model.step(sess)

            cnt = 1
            try:
                while True:
                    predictions_s, labels_s= model.step(sess)
                    hr_s, mrr_s, ndcg_s = evaluate(predictions_s, labels_s)
                    HR_s.append(hr_s)
                    MRR_s.append(mrr_s)
                    NDCG_s.append(ndcg_s)
                    cnt += 1
            except tf.errors.OutOfRangeError:
                hr_s = np.array(HR_s).mean()
                mrr_s = np.array(MRR_s).mean()
                ndcg_s = np.array(NDCG_s).mean()
                hr_s_list.append(hr_s)
                mrr_s_list.append(mrr_s)
                ndcg_s_list.append(ndcg_s)
                print("Epoch %d, finish testing " % epoch + "took: " +
                      time.strftime("%H: %M: %S", time.gmtime(time.time() - start_time)) + ';')
                print('Epoch %d, %s HR is %.4f, MRR is %.4f, NDCG is %.4f;' %
                      (epoch, name_s, hr_s, mrr_s, ndcg_s))
        print('=' * 30 + 'Finish training' + '=' * 30)
        print('%s best HR is %.4f, MRR is %.4f, NDCG is %.4f;' %
              (name_s, max(hr_s_list), max(mrr_s_list), max(ndcg_s_list)))

Building adjacent mats_s..
Get adjacent mats_s successfully.
Building adjacent mats_s..
Get adjacent mats_s successfully.
train_data <BatchDataset shapes: {user: (?,), item: (?,), label: (?,)}, types: {user: tf.int32, item: tf.int32, label: tf.int32}>
train_data除以batch_size 1249.84375
test_data <TensorSliceDataset shapes: {user: (), item: (), label: ()}, types: {user: tf.int32, item: tf.int32, label: tf.int32}>
self.layers_plus [64, 64, 64, 64]
Creating model with fresh parameters...
============================== EPOCH 1 ==============================
Epoch 1, step 300, with average loss of 0.9277 in last 300 steps;
Epoch 1, step 600, with average loss of 0.8560 in last 300 steps;
Epoch 1, step 900, with average loss of 0.8045 in last 300 steps;
Epoch 1, step 1200, with average loss of 0.7659 in last 300 steps;
Epoch 1, finish training took 00: 03: 11;
Epoch 1, finish testing took: 00: 01: 17;
Epoch 1, Amazon HR is 0.3681, MRR is 0.1715, NDCG is 0.2175;
============================== 

Epoch 16, step 19800, with average loss of 0.0333 in last 300 steps;
Epoch 16, finish training took 00: 03: 04;
Epoch 16, finish testing took: 00: 01: 15;
Epoch 16, Amazon HR is 0.4363, MRR is 0.2055, NDCG is 0.2597;
============================== EPOCH 17 ==============================
Epoch 17, step 20100, with average loss of 0.0322 in last 300 steps;
Epoch 17, step 20400, with average loss of 0.0207 in last 300 steps;
Epoch 17, step 20700, with average loss of 0.0255 in last 300 steps;
Epoch 17, step 21000, with average loss of 0.0304 in last 300 steps;
Epoch 17, finish training took 00: 03: 05;
Epoch 17, finish testing took: 00: 01: 14;
Epoch 17, Amazon HR is 0.4373, MRR is 0.2043, NDCG is 0.2590;
============================== EPOCH 18 ==============================
Epoch 18, step 21300, with average loss of 0.0288 in last 300 steps;
Epoch 18, step 21600, with average loss of 0.0179 in last 300 steps;
Epoch 18, step 21900, with average loss of 0.0239 in last 300 steps;
Epoch 18, 

KeyboardInterrupt: 